In [1]:
# GPT: Generatively Pre-trained Transformer

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-08 11:53:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.50MB/s    in 0.3s    

2024-07-08 11:53:03 (3.50 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print(len(text))

1115394


In [5]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [9]:
# unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [11]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # take string, output list of ints
decode = lambda l: ''.join([itos[i] for i in l]) # take list of ints, output string

In [12]:
# tradeoff: token size and vocab size

In [15]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [17]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [20]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [22]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [31]:
torch.manual_seed(1337)
batch_size = 4 # independent sequences that will be processed in parallel per forward/backward pass
block_size = 8 # maximum context length for predictions

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [39]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is {target}")

when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58
when input is [44, 53, 56, 1, 58] the target is 46
when input is [44, 53, 56, 1, 58, 46] the target is 39
when input is [44, 53, 56, 1, 58, 46, 39] the target is 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target is 1
when input is [52] the target is 58
when input is [52, 58] the target is 1
when input is [52, 58, 1] the target is 58
when input is [52, 58, 1, 58] the target is 46
when input is [52, 58, 1, 58, 46] the target is 39
w

In [124]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C): batch, time, channel

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # to comply with pytorch
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T] array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus only on last time step
            logits = logits[:, -1, :] # becomes [B, C]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # [B, C]
            # sample from distro
            idx_next = torch.multinomial(probs, num_samples=1) # [B, 1]
            # append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # [B, T+1]
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [125]:
model = BigramLanguageModel(vocab_size)

In [126]:
# create pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [127]:
# hyperparameters
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [128]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'eval']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [129]:
# training loop
for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['eval']:.4f}")
    # sample batch of data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.6036, val loss 4.5944
step 300: train loss 4.2593, val loss 4.2607
step 600: train loss 3.9636, val loss 3.9777
step 900: train loss 3.7171, val loss 3.7164
step 1200: train loss 3.5002, val loss 3.4995
step 1500: train loss 3.3065, val loss 3.3162
step 1800: train loss 3.1560, val loss 3.1619
step 2100: train loss 3.0375, val loss 3.0378
step 2400: train loss 2.9251, val loss 2.9345
step 2700: train loss 2.8529, val loss 2.8575


In [131]:
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=500)[0].tolist()))


Lq
chwndmirencre cope'$ATILUlx aTOUTa UNu mas&Lrt Yvy ty, ;
Bavgg y bblpe ICLe Ax?
O thong.
BUK'sUCBmane hDbuMmerd tysBys!TJUKdFOFo la,'BARst McasjoW touBUCHAowk WuiK3
Thwis
IXUt ny IlixDite EUK, MINabl, I
MEN LyRqRI muZveXEqulo, sty ur:
3Fut .
!
Athe y,a.
AItheAWAy mHcfuisagWhwhldit ftherd$otn?
THend, isg s?ichy oselopear.
GUy?
PTovX. stestOI pouI QYvo?y;
ARKigidisthy, it:Lfcailay'WInbeeLZke thwhiond p'd avQlBuso, sQfrb'mauawavZXWharereswn st.
hELke, hh Mg, lYRoikF:q
wlobulicrdwsj m ho ENaO&d;!
